In [1]:
import os, sys, gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
sys.path.append('/root/src')
from model_conv_1d import load_data, build_convolution1D_model, loss
file_path = '~/data/bitstampUSD_1-min_data_2012-01-01_to_2017-05-31.csv'

Using TensorFlow backend.


In [3]:
# data prepration
df = load_data(file_path)
df = df.drop(['Open', 'High', 'Low', 'Volume_(Currency)', 'Weighted_Price'], axis=1)
df['BTC_balance'] = 0
df['USD_balance'] = 1000
df['Portfolio_value'] = 0
df['total_rewards'] = 0

print (df.tail())

                             Close  Volume_(BTC)  BTC_balance  USD_balance  \
Timestamp                                                                    
2017-05-30 23:56:00+00:00  2181.37      1.700166            0         1000   
2017-05-30 23:57:00+00:00  2195.63      6.561029            0         1000   
2017-05-30 23:58:00+00:00  2191.83     15.662847            0         1000   
2017-05-30 23:59:00+00:00  2203.51     27.090309            0         1000   
2017-05-31 00:00:00+00:00  2208.33      9.961835            0         1000   

                           Portfolio_value  total_rewards  
Timestamp                                                  
2017-05-30 23:56:00+00:00                0              0  
2017-05-30 23:57:00+00:00                0              0  
2017-05-30 23:58:00+00:00                0              0  
2017-05-30 23:59:00+00:00                0              0  
2017-05-31 00:00:00+00:00                0              0  


In [4]:
# create the training, validation and test datasets
training_size = 1000000
validation_size = 500000
test_size = 150904
train_df = df[:training_size]
valid_df = df[training_size:training_size+validation_size]
test_df = df[training_size+validation_size:]
print(len(train_df), len(valid_df), len(test_df))

1000000 500000 150904


In [5]:
# model's hyperparameters
window_size = 60
nb_filter = 8
filter_length = 5
nb_input_series = 6
nb_outputs = 2
batch_size = 480
epochs = 5

In [6]:
# build the model and print the summary of the model
model = build_convolution1D_model(window_size=window_size, filter_length=filter_length, nb_input_series=nb_input_series, nb_outputs=nb_outputs, nb_filter=nb_filter)
print('\n\nModel with input size {}, output size {}, {} conv filters of length {}'.format(model.input_shape, model.output_shape, nb_filter, filter_length))
model.summary()



Model with input size (None, 60, 6), output size (None, 2), 8 conv filters of length 5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 56, 8)             248       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 28, 8)             0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 24, 8)             328       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 12, 8)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 96)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 194       
Total params: 770
Trainable params: 770
Non-trainable